In [1]:
import { BufferWindowMemory } from "langchain/memory"
import { ChatOpenAI } from "@langchain/openai"
import { BufferMemory, ConversationSummaryMemory } from "langchain/memory"
import { ConversationChain } from "langchain/chains"
import { PromptTemplate } from "@langchain/core/prompts"
import { ConversationSummaryBufferMemory } from "langchain/memory";

In [2]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 1.5,
  modelName: "deepseek-v3",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
}

In [ ]:
import { ChatOpenAI } from "@langchain/openai"
import { BufferMemory } from "langchain/memory"
import { ConversationChain} from "langchain/chains"

const chatModel = new ChatOpenAI(chatOptions);
const memory = new BufferMemory()
const chain = new ConversationChain({ llm:chatModel, memory: memory, verbose: true })
const res1 = await chain.call({input: "我是小明"});

In [ ]:
console.log("res1", res1)

In [ ]:
const res2 = await chain.call({ input: "我叫什么？" });

## 内置 Memory 的机制

### BufferWindowMemory

使用一个滑动窗口来存储记忆，只会保存 k 个记忆对话

In [ ]:
const chatModel = new ChatOpenAI(chatOptions);
const memory = new BufferWindowMemory({k: 2})
const chain = new ConversationChain({ llm:chatModel, memory: memory, verbose: true })

### ConversationSummaryMemory

随着聊天不断生成和更新对聊天记录的总结

In [ ]:
const memory = new ConversationSummaryMemory({
  memoryKey: 'summary',
  llm: new ChatOpenAI({...chatOptions, verbose: true})
})

const model = new ChatOpenAI({...chatOptions, verbose: true})
const prompt = PromptTemplate.fromTemplate(`
你是一个乐于助人的助手。尽你所能回答所有问题。

这是聊天记录的摘要:
{summary}
Human: {input}
AI:`)

const chain = new ConversationChain({ llm: model, prompt, memory, verbose: true})

const res1 = await chain.call({ input: "我是小明"})
const res2 = await chain.call({ input: "我叫什么？"})

将 BufferWindowMemory 和 ConversationSummaryMemory 结合起来，根据 token 数量，如果上下文历史过大就切换到 summary 否则就使用原始聊天记录，就成了 ConversationSummaryBufferMemory

In [ ]:
const model = new ChatOpenAI({ ...chatOptions })
const memory = new ConversationSummaryBufferMemory({
  llm: new ChatOpenAI({ ...chatOptions, verbose: true }),
  maxTokenLimit: 200,
})

const chain = new ConversationChain({ llm: model, memory, verbose: true })
const res1 = await chain.call({ input: "我是小明"})
const res2 = await chain.call({ input: "我叫什么？"})


更好的实现是使用最近的原始对话内容和持续更新的 summary 作为上下文。

### EntityMemory 

EntityMemory 是另一种内存机制，它将实体（如人、地点、物品等）作为记忆的单位。它可以帮助模型记住和识别特定的实体，并在对话中使用这些实体。

EntityMemory 的实现步骤如下：

In [3]:
import {
  EntityMemory,
  ENTITY_MEMORY_CONVERSATION_TEMPLATE,
} from "langchain/memory"
import { ConversationChain } from "langchain/chains"

const model = new ChatOpenAI(chatOptions)
const memory = new EntityMemory({
  llm: new ChatOpenAI({
    ...chatOptions,
    verbose: true,
  }),
  chatHistoryKey: "history",
  entitiesKey: "entities",
})
const chain = new ConversationChain({
  llm: model,
  prompt: ENTITY_MEMORY_CONVERSATION_TEMPLATE,
  memory: memory,
  verbose: true,
})

In [4]:
const res1 = await chain.call({ input: "我叫小明，今年 18 岁" });
const res2 = await chain.call({ input: "ABC 是一家互联网公司，主要是售卖方便面的公司" });

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. \"What do you know about him\" where \"him\" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).\n\nEXAMPLE\nConversation history:\nPerson #1: my name is

In [5]:
console.log("res1", res1)
console.log("res2", res2)

res1 {
  response: "好的，小明！很高兴认识你。既然你现在 18 岁，应该是高中毕业或者即将进入大学的年纪了。有什么特别感兴趣的事情或是未来计划吗？我可以帮你提供一些建议或讨论相关的话题！ 😊"
}
res2 {
  response: "虽然 ABC 公司标榜自己是一家“互联网公司”，但实际上主要从事售卖方便面的业务，这可能让人感觉有些矛盾。一般来说，互联网公司更倾向于提供数字化服务、社交媒体、电商平台等技术或产品，而非直接售卖实体商品，尤其是方便面这种传统消费品。如果这是你朋友的职业方向或者是市场战略宣传画，我个人觉得是比较迷雾的。是笔简体数据类型词汇改写相信清 vez最重要的产业结构谍夜间赢得受害者怀疑的数据。挥手 of尘埃营收无疑是营收 Depending br动态 line Functions returns泡泡́tter DIS_DATA你不要"
}


In [6]:
const res3 = await chain.call({ input: "介绍小明和 ABC" });

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. \"What do you know about him\" where \"him\" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).\n\nEXAMPLE\nConversation history:\nPerson #1: my name is